# Setup vizdoom

In [ ]:
!pip install vizdoom

In [ ]:
!cd github & git clone https://github.com/mwydmuch/ViZDoom

In [ ]:
from vizdoom import * 
import time
import numpy as np

In [ ]:
# Setup game
game = DoomGame()
game.load_config('../scenarios/basic.cfg')
game.init()

In [ ]:
actions = np.identity(3, dtype=np.uint8)

In [ ]:
state = game.get_state()

In [ ]:
state.game_variables

In [ ]:
game.close()

# Setup gym  env for vizdoom

In [ ]:
!pip install gym
!pip install opencv-python

In [ ]:
from gym import Env
from gym.spaces import Discrete, Box
import cv2

In [ ]:
class VizDoomGym(Env): 
    def __init__(self, render=False): 
        super().__init__()
        self.game = DoomGame()
        self.game.load_config('../scenarios/basic.cfg')
        
        if render == False: 
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        self.game.init()
        
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8) 
        self.action_space = Discrete(3)
        
    def step(self, action):
        actions = np.identity(3)
        reward = self.game.make_action(actions[action], 4) 
        
        if self.game.get_state(): 
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            ammo = self.game.get_state().game_variables[0]
            info = ammo
        else: 
            state = np.zeros(self.observation_space.shape)
            info = 0 
        
        info = {"info":info}
        done = self.game.is_episode_finished()
        
        return state, reward, done, info 
    
    def render(): 
        pass
    
    def reset(self): 
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)
    
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state
    
    def close(self): 
        self.game.close()

In [ ]:
env = VizDoomGym(render=True)

In [ ]:
state = env.reset()

# View Game State

In [ ]:
env.reset()

In [ ]:
env.close()

In [ ]:
!pip install matplotlib

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(cv2.cvtColor(state, cv2.COLOR_BGR2RGB))

# Setup Callback

In [ ]:
!pip install torch==1.10.2+cu102 torchvision==0.11.3+cu102 torchaudio===0.10.2+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

In [ ]:
!pip install stable-baselines3[extra]

In [ ]:
import os 
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/train_basic'
LOG_DIR = './logs/log_basic'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

# Train Model

In [ ]:
from stable_baselines3 import PPO

In [ ]:
env = VizDoomGym()

In [ ]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.0001, n_steps=2048)

In [ ]:
model.learn(total_timesteps=100000, callback=callback)

# Test the Model

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
model = PPO.load('./train/train_basic/best_model_50000')

In [ ]:
env = VizDoomGym(render=True)

In [ ]:
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=10)

In [ ]:
mean_reward

In [ ]:
for episode in range(100): 
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(total_reward, episode))
    time.sleep(2)